In [1]:
import sys

from youtube_transcript_api import YouTubeTranscriptApi

from pytube import YouTube

def save_transcript_to_file(video_id, transcript):

    filename = f"{video_id}_transcript.txt"

    with open(filename, 'w', encoding='utf-8') as file:

        for entry in transcript:

            file.write(f"{entry['start']} - {entry['start'] + entry['duration']}: {entry['text']}\n")

    print(f"Transcript saved to {filename}")



def process_video_url(video_url):

    video_id = video_url.split('v=')[-1].split('&')[0]

   

    try:

        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

        manual_transcript = None

        for transcript in transcript_list:

            if not transcript.is_generated:

                manual_transcript = transcript.fetch()

                break

        if manual_transcript is None:

            manual_transcript = transcript_list.find_generated_transcript(['en']).fetch()

       

        save_transcript_to_file(video_id, manual_transcript)

        yt = YouTube(video_url)

        video_stream = yt.streams.filter(progressive=True, file_extension="mp4").order_by("resolution").desc().first()

        video_filename = video_stream.download(filename=f"{yt.title}_video")

        audio_stream = yt.streams.filter(only_audio=True).first()

        audio_filename = audio_stream.download(filename=f"{yt.title}_audio")

        print(f"Video saved to {video_filename}")

        print(f"Audio saved to {audio_filename}")

    except Exception as e:

        print(f"An error occurred: {e}")



def process_urls_from_file(file_path):

    try:

        with open(file_path, 'r') as file:

            urls = file.read().splitlines()

       

        for url in urls:

            process_video_url(url)

    except Exception as e:

        print(f"An error occurred while processing the URLs: {e}")



if __name__ == "__main__":

    if len(sys.argv) < 2:

        print("Usage: python script_name.py <file_with_youtube_urls>")

    else:

        urls_file = sys.argv[1]

        process_urls_from_file(urls_file)

An error occurred while processing the URLs: [Errno 2] No such file or directory: '-f'


In [14]:
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube
import os

def get_video_info(video_url):
    video_id = video_url.split('v=')[1]
    try:
        video_info = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        return video_info
    except Exception as e:
        print(f"Error retrieving transcript for {video_url}: {e}")
        return None

def get_video_title(video_url):
    video = YouTube(video_url)
    return video.title

def create_text_files(video_links, folder_name='youtube_text_scripts'):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for video_url in video_links:
        video_info = get_video_info(video_url)
        if video_info is not None:
            title = get_video_title(video_url)
            keywords = [keyword.lower() for keyword in title.split()]

            # Create a folder for storing transcripts
            folder_path = os.path.join(folder_name, '_'.join(keywords))
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # Write auto-generated captions to the text file
            file_path = os.path.join(folder_path, 'script.txt')
            with open(file_path, 'w', encoding='utf-8') as file:
                captions = [entry['text'] for entry in video_info]
                file.write('\n'.join(captions))

if __name__ == "__main__":
    # Read all links from youtube_urls.txt
    with open('youtube_urls.txt', 'r') as file:
        all_links = [line.strip() for line in file.readlines()]

    create_text_files(all_links)


Error retrieving transcript for https://www.youtube.com/watch?v=FgGW-ivQ8z4: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=FgGW-ivQ8z4! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Error retrieving transcript for https://www.youtube.com/watch?v=Xj1CtGvv-DE: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=Xj1CtGvv-DE! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be ret

Error retrieving transcript for https://www.youtube.com/watch?v=GyKH3EzSSZw: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=GyKH3EzSSZw! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Error retrieving transcript for https://www.youtube.com/watch?v=foN5ULb_3qo: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=foN5ULb_3qo! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be ret

Error retrieving transcript for https://www.youtube.com/watch?v=4zEoooRGTQ4: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=4zEoooRGTQ4! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Error retrieving transcript for https://www.youtube.com/watch?v=CJa39-fehJA: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=CJa39-fehJA! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be ret

In [6]:
pip install pytube


Defaulting to user installation because normal site-packages is not writeable
  Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import yt_dlp

def get_playlist_video_links(playlist_url, max_videos=None):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'force_generic_extractor': True,
        'youtube_include_dash_manifest': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(playlist_url, download=False)
            entries = result.get('entries', [])
            video_links = [entry['url'] for entry in entries if not entry.get('is_live', False) and entry.get('url', '').startswith('http')]
            
            # Optional: Limit the number of videos
            if max_videos is not None:
                video_links = video_links[:max_videos]
                
        except Exception as e:
            print(f"Error: {e}")
            video_links = []

    return video_links

playlist_url = "https://www.youtube.com/playlist?list=PLBEIBBdgAOArw0ljqpJoywBxXtSrkqVCt"
video_links = get_playlist_video_links(playlist_url, max_videos=None)

with open('youtube_urls.txt', 'w') as file:
    for link in video_links:
        file.write(link + '\n')


In [20]:
import yt_dlp

def get_channel_playlists(channel_url):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'force_generic_extractor': True,
        'youtube_include_dash_manifest': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(channel_url, download=False)
            playlists = [entry['url'] for entry in result.get('entries', []) if entry['ie_key'] == 'YoutubeTab']
        except Exception as e:
            print(f"Error: {e}")
            playlists = []

    return playlists

if __name__ == "__main__":
    channel_url = "https://www.youtube.com/@SoftWhiteUnderbelly/playlists"
    playlists = get_channel_playlists(channel_url)

    with open("youtube_urls.txt", "w") as file:
        for playlist_url in playlists:
            file.write(playlist_url + "\n")

    print("Playlist URLs written to youtube_urls.txt")


Playlist URLs written to youtube_urls.txt


In [ ]:
import yt_dlp
import os

def get_playlist_video_links(playlist_url):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'force_generic_extractor': True,
        'youtube_include_dash_manifest': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(playlist_url, download=False)
            video_links = [entry['url'] for entry in result.get('entries', [])]
        except Exception as e:
            print(f"Error: {e}")
            video_links = []

    return video_links

def create_text_files(video_links, folder_name):
    for video_url in video_links:
        video_info = get_video_info(video_url)
        title = video_info.get('title', '')
        keywords = [keyword.lower() for keyword in title.split()]
        folder_path = os.path.join(folder_name, '_'.join(keywords))

        # Create the folder if it doesn't exist
        os.makedirs(folder_path, exist_ok=True)

        # Write auto-generated captions to the text file
        file_path = os.path.join(folder_path, 'script.txt')
        auto_captions_list = video_info.get('automatic_captions', [])
        auto_captions = auto_captions_list[0].get('content', '') if auto_captions_list else ''
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(auto_captions)


def get_video_info(video_url):
    ydl_opts = {
        'quiet': True,
        'writeautomaticsub': True,
        'force_generic_extractor': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(video_url, download=False)
            automatic_captions = result.get('automatic_captions', {}).get('en', [])
            return {'title': result.get('title', ''), 'automatic_captions': automatic_captions}
        except Exception as e:
            print(f"Error: {e}")
            return {'title': '', 'automatic_captions': []}




def process_playlists(playlists_url):
    with open(playlists_url, 'r') as file:
        playlists = file.readlines()

    for playlist_url in playlists:
        playlist_url = playlist_url.strip()
        print(f"Processing playlist: {playlist_url}")

        video_links = get_playlist_video_links(playlist_url)
        folder_name = 'softwhite_playlists'
        create_text_files(video_links, folder_name)

if __name__ == "__main__":
    playlists_url = "youtube_urls.txt"
    process_playlists(playlists_url)


Processing playlist: https://www.youtube.com/playlist?list=PLBEIBBdgAOArK19RXJ-DUoKQB592eTyDA


Processing playlist: https://www.youtube.com/playlist?list=PLBEIBBdgAOArodQ5aGViY04m1Xu0Wms8C


ERROR: [youtube] G3WjpS5elKw: Private video. Sign in if you've been granted access to this video


Error: ERROR: [youtube] G3WjpS5elKw: Private video. Sign in if you've been granted access to this video


ERROR: [youtube] qlqIhtq0Lc8: Private video. Sign in if you've been granted access to this video


...

Error: ERROR: [youtube] eW_nsGWWON8: Video unavailable. This video has been removed by the uploader


ERROR: [youtube] z-B_2uo6MsQ: Private video. Sign in if you've been granted access to this video


Error: ERROR: [youtube] z-B_2uo6MsQ: Private video. Sign in if you've been granted access to this video


ERROR: [youtube] awWoVNRz3jA: Private video. Sign in if you've been granted access to this video


Error: ERROR: [youtube] awWoVNRz3jA: Private video. Sign in if you've been granted access to this video


import yt_dlp
import os

def get_playlist_video_links(playlist_url, max_videos=5):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'force_generic_extractor': True,
        'youtube_include_dash_manifest': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(playlist_url, download=False)
            video_links = [entry['url'] for entry in result.get('entries', [])[:max_videos]]
        except Exception as e:
            print(f"Error: {e}")
            video_links = []

    return video_links

def create_text_files(video_links, folder_name):
    os.makedirs(folder_name, exist_ok=True)

    for video_url in video_links:
        try:
            video_info = yt_dlp.YoutubeDL().extract_info(video_url, download=False)
            title = video_info.get('title', '')
            keywords = [keyword.lower() for keyword in title.split()]
            file_name = '_'.join(keywords) + '.txt'
            file_path = os.path.join(folder_name, file_name)

            # Write auto-generated captions to the text file
            with open(file_path, 'w', encoding='utf-8') as file:
                auto_captions = video_info.get('automatic_captions', {}).get('en', {}).get('content', '')
                file.write(auto_captions)

            print(f"Script downloaded for video: {title}")
        except Exception as e:
            print(f"Error downloading script for video {video_url}: {e}")

if __name__ == "__main__":
    playlist_url = "https://www.youtube.com/playlist?list=PLBEIBBdgAOArw0ljqpJoywBxXtSrkqVCt"
    video_links = get_playlist_video_links(playlist_url, max_videos=5)
    create_text_files(video_links, folder_name="youtube_text_scripts")


In [70]:
import os
import yt_dlp

def sanitize_filename(name):
    """
    Sanitize the filename by replacing or removing invalid characters.
    """
    return "".join([c if c.isalnum() or c in " _-." else "_" for c in name])

def get_playlist_video_links(playlist_url):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'force_generic_extractor': True,
        'youtube_include_dash_manifest': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(playlist_url, download=False)
            video_links = [entry['url'] for entry in result.get('entries', [])]
            return video_links
        except Exception as e:
            print(f"Error: {e}")
            return []

def get_video_info(video_url):
    ydl_opts = {
        'quiet': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en'],
        'skip_download': True,
        'force_generic_extractor': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(video_url, download=False)
            if 'automatic_captions' in result and 'en' in result['automatic_captions']:
                automatic_captions = result['automatic_captions']['en'][0].get('content', '')
                return {'title': result.get('title', ''), 'automatic_captions': automatic_captions}
            else:
                return {'title': result.get('title', ''), 'automatic_captions': ''}
        except Exception as e:
            print(f"Error: {e}")
            return {'title': '', 'automatic_captions': ''}

def create_text_files(video_links, folder_name):
    for video_url in video_links:
        video_info = get_video_info(video_url)
        title = video_info.get('title', '')
        keywords = [sanitize_filename(keyword.lower()) for keyword in title.split()]
        folder_path = os.path.join('softwhite_playlists', folder_name, '_'.join(keywords))
        os.makedirs(folder_path, exist_ok=True)

        file_path = os.path.join(folder_path, 'script.txt')
        auto_captions = video_info.get('automatic_captions', '')

        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(auto_captions)

def get_playlist_title(playlist_url):
    ydl_opts = {
        'quiet': True,
        'extract_flat': 'in_playlist',
        'force_generic_extractor': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(playlist_url, download=False)
            return sanitize_filename(result.get('title', 'unknown_playlist'))
        except Exception as e:
            print(f"Error getting playlist title: {e}")
            return 'unknown_playlist'

def process_playlists(playlists_url):
    with open(playlists_url, 'r') as file:
        playlists = file.readlines()

    for playlist_url in playlists:
        playlist_url = playlist_url.strip()
        playlist_title = get_playlist_title(playlist_url)
        video_links = get_playlist_video_links(playlist_url)
        create_text_files(video_links, playlist_title)

if __name__ == "__main__":
    playlists_url = "youtube_urls.txt"
    process_playlists(playlists_url)


KeyboardInterrupt: 